In [1]:
!pip install transformers accelerate
!pip install transformers[sentencepiece]
!pip install sentencepiece
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
!pip install datasets
from datasets import load_metric
%pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv
!pip install openprompt
import os
import csv
from openprompt.data_utils import InputExample

from datasets import Dataset, load_metric
from sklearn import preprocessing
# notebook_login()
from tqdm import tqdm
CUDA_LAUNCH_BLOCKING=1


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
train_I = pd.read_csv(r'/content/drive/MyDrive/dataset I/train.csv', encoding='utf_8_sig')
train_I.fillna('', inplace=True)
train_II = pd.read_csv(r'/content/drive/MyDrive/dataset II/train.csv', encoding='utf_8_sig')
train_II.fillna('', inplace=True)
train = pd.concat([train_I,train_II],axis=0)
test_I = pd.read_csv(r'/content/drive/MyDrive/dataset II/test.csv', encoding='utf_8_sig')
test_I.fillna('', inplace=True)
test_II = pd.read_csv(r'/content/drive/MyDrive/dataset II/test.csv', encoding='utf_8_sig')
test_II.fillna('', inplace=True)

In [25]:
# unified columns
train = train[['label','text']].reset_index(drop=True)
label2id = {'positive':0,'negative':1,'Adaptive':2,'Perfective':3,'Corrective':4}
train = train.replace({'label':label2id})
test_I = test_I.replace({'label':label2id})
test_II = test_II.replace({'label':label2id})

In [5]:
len(train)

8333

In [6]:
train

,label,text
0,0,XSS 취약점 수정
1,1,"Auto merge of #4754 - daira:fix-enable-debug, ..."
2,0,CAMEL-9309: Make it easier to turn on|off java...
3,0,Fixed bug #72627: Memory Leakage In exif_proce...
4,1,Merge branch 'dev'
...,...,...
8328,2,glib-2.0: add GLib.HashTable.foreach\n\nFixes ...
8329,2,"search is more complete, support forward/rever..."
8330,4,"Extract Function: Consider reference ""broken"" ..."
8331,3,First part of https://jira.duraspace.org/brows...


In [ ]:
test

In [27]:
from openprompt.data_utils import InputExample

dataset = {}
# for split in ['train','test']:
    # dataset[split] = []
dataset['train'] = []
dataset['test'] = []
for index, row in train.iterrows():
# print(row.text)
# print(row['text'])
  input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
  dataset['train'].append(input_example)
for index, row in test_II.iterrows():
    # print(row.text)
    # print(row['text'])
    input_example = InputExample(text_a  =  row['text'], label=int(row['label']))
    dataset['test'].append(input_example)
print(dataset['train'][0])

{
  "guid": null,
  "label": 0,
  "meta": {},
  "text_a": "XSS \ucde8\uc57d\uc810 \uc218\uc815",
  "text_b": "",
  "tgt_text": null
}



In [8]:
len(train)

8333

In [26]:
len(test_II)

538

In [10]:
from openprompt.plms import load_plm

plm, tokenizer, model_config, WrapperClass = load_plm("t5", "t5-base")


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available 

In [11]:
# Constructing Template
# A template can be constructed from the yaml config, but it can also be constructed by directly passing arguments.
from openprompt.prompts import ManualTemplate
template_text = 'This {"placeholder":"text_a"} is a {"mask"} commit.'
mytemplate = ManualTemplate(tokenizer=tokenizer, text=template_text)

# To better understand how does the template wrap the example, we visualize one instance.

wrapped_example = mytemplate.wrap_one_example(dataset['train'][0])
print(wrapped_example)

[[{'text': 'This', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': ' XSS 취약점 수정', 'loss_ids': 0, 'shortenable_ids': 1}, {'text': ' is a', 'loss_ids': 0, 'shortenable_ids': 0}, {'text': '<mask>', 'loss_ids': 1, 'shortenable_ids': 0}, {'text': ' commit.', 'loss_ids': 0, 'shortenable_ids': 0}], {'label': 0}]


In [12]:
wrapped_t5tokenizer = WrapperClass(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")
# or
from openprompt.plms import T5TokenizerWrapper
wrapped_t5tokenizer= T5TokenizerWrapper(max_seq_length=128, decoder_max_length=3, tokenizer=tokenizer,truncate_method="head")

# You can see what a tokenized example looks like by
tokenized_example = wrapped_t5tokenizer.tokenize_one_example(wrapped_example, teacher_forcing=False)
print(tokenized_example)
print(tokenizer.convert_ids_to_tokens(tokenized_example['input_ids']))
print(tokenizer.convert_ids_to_tokens(tokenized_example['decoder_input_ids']))


{'input_ids': [100, 3, 4, 4256, 3, 2, 3, 2, 19, 3, 9, 32099, 10042, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [0, 32099, 0], 'loss_ids': [0, 1, 0]}
['▁This', '▁', 'X', 'SS', '▁', '<unk>', '▁', '<unk>', '▁is', '▁', 'a', '<extra_id_0>', '▁commit', '.', '</s>', '<pad>', '<p

In [13]:
model_inputs = {}
for split in ['train', 'test']:
    model_inputs[split] = []
    for sample in dataset[split]:
        tokenized_example = wrapped_t5tokenizer.tokenize_one_example(mytemplate.wrap_one_example(sample), teacher_forcing=False)
        model_inputs[split].append(tokenized_example)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


In [14]:
# We provide a `PromptDataLoader` class to help you do all the above matters and wrap them into an `torch.DataLoader` style iterator.
from openprompt import PromptDataLoader

train_dataloader = PromptDataLoader(dataset=dataset["train"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=True, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")
# next(iter(train_dataloader))

tokenizing: 8333it [00:12, 654.45it/s]


In [15]:
from openprompt.prompts import ManualVerbalizer,KnowledgeableVerbalizer
import torch

# for example the verbalizer contains multiple label words in each class
# myverbalizer = ManualVerbalizer(tokenizer, num_classes=3,
#                         label_words=[["Adaptive"], ["Perfective"], ["Corrective"]])
myverbalizer = KnowledgeableVerbalizer(tokenizer, num_classes=5).from_file("/content/drive/MyDrive/knowledgeable_verbalizer_allset.txt")

print(myverbalizer.label_words_ids)
logits = torch.randn(2,len(tokenizer)) # creating a pseudo output from the plm, and
print(myverbalizer.process_logits(logits)) # see what the verbalizer do


##Num of label words for each label: [21, 20, 19, 20, 20]
Parameter containing:
tensor([[[ 2451,     0,     0,     0,     0],
         [ 3614,     0,     0,     0,     0],
         [ 1346,     0,     0,     0,     0],
         [  766,     0,     0,     0,     0],
         [ 7992,     0,     0,     0,     0],
         [ 1006,    35,     0,     0,     0],
         [    3, 11005,   524,     0,     0],
         [   16,  4334,     0,     0,     0],
         [    3, 21477,     0,     0,     0],
         [   36,  5595,     0,     0,     0],
         [   16, 12388,   687,   179,     0],
         [    3, 23094,     0,     0,     0],
         [  417,     0,     0,     0,     0],
         [ 3442,     0,     0,     0,     0],
         [  143,     0,     0,     0,     0],
         [ 2210,     0,     0,     0,     0],
         [ 1669,     0,     0,     0,     0],
         [11892,    15,     0,     0,     0],
         [   21,    17,  4921,     0,     0],
         [ 4095,     0,     0,     0,     0],


In [16]:
from openprompt import PromptForClassification

use_cuda = True
prompt_model = PromptForClassification(plm=plm,template=mytemplate, verbalizer=myverbalizer, freeze_plm=False)
if use_cuda:
    prompt_model=  prompt_model.cuda()


In [17]:
# Now the training is standard
from transformers import  AdamW, get_linear_schedule_with_warmup
loss_func = torch.nn.CrossEntropyLoss()
no_decay = ['bias', 'LayerNorm.weight']
# it's always good practice to set no decay to biase and LayerNorm parameters
optimizer_grouped_parameters = [
    {'params': [p for n, p in prompt_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in prompt_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
# Set up early stopping parameters
best_validation_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
waited_epochs = 0
# Evaluate
validation_dataloader = PromptDataLoader(dataset=dataset["test"], template=mytemplate, tokenizer=tokenizer,
    tokenizer_wrapper_class=WrapperClass, max_seq_length=256, decoder_max_length=3,
    batch_size=4,shuffle=False, teacher_forcing=False, predict_eos_token=False,
    truncate_method="head")

tokenizing: 538it [00:00, 543.28it/s]


In [20]:
for epoch in range(10):
    tot_loss = 0
    for step, inputs in enumerate(train_dataloader):
        if use_cuda:
            inputs = inputs.cuda()
        logits = prompt_model(inputs)
        labels = inputs['label']
        loss = loss_func(logits, labels)
        loss.backward()
        tot_loss += loss.item()
        optimizer.step()
        optimizer.zero_grad()
        if step % 100 == 1:
            print("Epoch {}, average loss: {}".format(epoch, tot_loss / (step + 1)), flush=True)

    # Validation loop
    validation_loss = 0
    prompt_model.eval()
    with torch.no_grad():
        for val_step, val_inputs in enumerate(validation_dataloader):
            if use_cuda:
                val_inputs = val_inputs.cuda()
            val_logits = prompt_model(val_inputs)
            val_labels = val_inputs['label']
            val_loss = loss_func(val_logits, val_labels)
            validation_loss += val_loss.item()

    validation_loss /= (val_step + 1)
    print("Epoch {}, validation loss: {}".format(epoch, validation_loss), flush=True)

    # Check for early stopping
    if validation_loss < best_validation_loss:
        best_validation_loss = validation_loss
        waited_epochs = 0
        # Save the model's state dictionary to a file
        torch.save(prompt_model.state_dict(), "best_model_state_dict.pt")
    else:
        waited_epochs += 1
        if waited_epochs >= patience:
            print("Early stopping at epoch {}.".format(epoch), flush=True)
            break

    prompt_model.train()  # Set the model back to training mode

Epoch 0, average loss: 2.08571857213974
Epoch 0, average loss: 1.0788176308075588
Epoch 0, average loss: 0.9914582099507351
Epoch 0, average loss: 0.9302410507167609
Epoch 0, average loss: 0.8685847764176813
Epoch 0, average loss: 0.8155660248895833
Epoch 0, average loss: 0.7725620804821757
Epoch 0, average loss: 0.7316511963398602
Epoch 0, average loss: 0.7069338795571052
Epoch 0, average loss: 0.6829963026314296
Epoch 0, average loss: 0.6603745563585913
Epoch 0, average loss: 0.6529162172236549
Epoch 0, average loss: 0.6356623573771084
Epoch 0, average loss: 0.6228108287398373
Epoch 0, average loss: 0.6109687105590443
Epoch 0, average loss: 0.6046122868208044
Epoch 0, average loss: 0.5964951565653114
Epoch 0, average loss: 0.5858002445796235
Epoch 0, average loss: 0.5804118297308823
Epoch 0, average loss: 0.5741048949147789
Epoch 0, average loss: 0.5692019621705772
Epoch 0, validation loss: 0.3175298324362798
Epoch 1, average loss: 0.2622385323047638
Epoch 1, average loss: 0.27970669

In [29]:
from sklearn.metrics import classification_report
allpreds = []
alllabels = []
for step, inputs in enumerate(validation_dataloader):
    if use_cuda:
        inputs = inputs.cuda()
    logits = prompt_model(inputs)
    labels = inputs['label']
    alllabels.extend(labels.cpu().tolist())
    allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())

# Generate the classification report
class_names = [f"Class_{i}" for i in range(3)]  # Replace num_classes with the number of classes in your dataset
classification_rep = classification_report(alllabels, allpreds, digits=4)

print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000         0
           1     0.0000    0.0000    0.0000         0
           2     0.7865    0.7910    0.7887       177
           3     0.8421    0.7111    0.7711       180
           4     0.8208    0.7845    0.8023       181

    accuracy                         0.7621       538
   macro avg     0.4899    0.4573    0.4724       538
weighted avg     0.8167    0.7621    0.7874       538



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
